# 전략을 위한 Code

In [1]:
from Backtest_source import *

In [6]:
class Strategy_Volatility_new(Strategy):
    def __init__(self, data_d, data_hr, strategy_list=['base'], trading_fee=0.1):
        self.data_d = data_d
        self.data_hr = data_hr
#         self.name = name
        self.strategy_list = strategy_list
        self.trading_fee = trading_fee
    
    def input_buy_signal_vol_d(self,data): # buy_signal, open_adj
        data_hr = data.copy()
        data_hr.index.name = None
        dict_ohlcv = {'open':'first','high':'max','low':'min','close':'last','value':'sum'}
        data_period = data_hr.resample('24H', origin=data_hr.index[0].replace(hour=9) ).apply(dict_ohlcv)
        # target_price 넣기
        price_range = data_period['high']-data_period['low']
        k_value = np.maximum( 0.5, ( abs( data_period.open.shift(1) - data_period.close.shift(1) ) / price_range.shift(1) ))
#         k_value = 0.5
        buy_price = data_period['open'] + price_range.shift(1) * k_value
        data_period['buy_price'] = buy_price
        # 시간별 자료에 넣기
        data_hr['buy_price'] = data_period['buy_price']  #9시에 들어감.
        data_hr['buy_price'].fillna(method='ffill', inplace=True)
       # signal 넣기
        data_hr['buy_signal'] = np.where((data_hr['high'] > data_hr['buy_price'])&(data_hr['open']<data_hr['buy_price']), 1, np.nan)
        # 수익률적용위해 매수시점의 수익률은 open_adj로 입력 -> return계산할때 넣는걸로 변경
        data_hr['open_adj'] = np.where(data_hr['buy_signal']==1, data_hr['buy_price'], np.nan)
        data['buy_price'] = data_hr['buy_price']
        data['buy_signal'] = data_hr['buy_signal']
        data['open_adj'] = data_hr['open_adj']
        return data
    
    def input_buy_signal_vol_m(self,data, volbreak_daily=0.2): # buy_signal, open_adj
        data_hr = data.copy()
        data_hr.index.name = None
        lines_per_day = len(data_hr.loc[:data_hr.index[0]+relativedelta(days=1)])-1
        data_hr['open_prd'] = data_hr['open'].shift(lines_per_day-1)
        data_hr['high_prd'] = data_hr['high'].rolling(lines_per_day).max()
        data_hr['low_prd'] = data_hr['low'].rolling(lines_per_day).min()
        data_hr['close_prd'] = data_hr['close'].shift(1)
        k_value = np.maximum(0.5, ( abs( data_hr.open_prd.shift(1) - data_hr.close_prd.shift(1) ) / (data_hr.high_prd.shift(1)-data_hr.low_prd.shift(1)) ))
        data_hr['buy_price'] = data_hr.open + (data_hr.high_prd.shift(1)-data_hr.low_prd.shift(1)) * k_value
        # 10분봉으로 하면 buy_p대상이 24x6개. 
        eval_no = int(lines_per_day*volbreak_daily)
        data_hr['buy_price'] = [find_n_max(buy_p[np.array(buy_p)>open_p], eval_no) if len(buy_p[np.array(buy_p)>open_p]) >=eval_no else 0 for buy_p,open_p in zip(data_hr['buy_price'].rolling(lines_per_day), data_hr['open'])]

       # signal 넣기
        data_hr['buy_signal'] = np.where((data_hr['high'] > data_hr['buy_price'])&(data_hr['open']<data_hr['buy_price']), 1, np.nan)
        # 수익률적용위해 매수시점의 수익률은 open_adj로 입력 -> return계산할때 넣는걸로 변경
        data_hr['open_adj'] = np.where(data_hr['buy_signal']==1, data_hr['buy_price'], np.nan)
        data['buy_price'] = data_hr['buy_price']
        data['buy_signal'] = data_hr['buy_signal']
        data['open_adj'] = data_hr['open_adj']
        return data
    
    def input_buy_signal_supertrend(self, data, rolling_timeframe=10, multiplier=3):
        data_hr = data.copy()
#         rolling_timeframe = 10
#         multiplier = 3
        data_hr['tr'] = data_hr['high'] - data_hr['low']  
#         data_hr['tr'] = np.max(data_hr['close'].shift(1), data_hr['high']) - np.min(data_hr['close'].shift(1), data_hr['low'])  
        data_hr['atr'] = data_hr['tr'].rolling(rolling_timeframe).mean()
        data_hr['buy_price'] = (data_hr['high'].shift(1) + data_hr['low'].shift(1))/2 + data_hr['atr'].shift(1)*multiplier
       # signal 넣기
        data_hr['buy_signal'] = np.where((data_hr['high'] > data_hr['buy_price'])&(data_hr['open']<data_hr['buy_price']), 1, np.nan)
        # 수익률적용위해 매수시점의 수익률은 open_adj로 입력 -> return계산할때 넣는걸로 변경
        data_hr['open_adj'] = np.where(data_hr['buy_signal']==1, data_hr['buy_price'], np.nan)
        data['buy_price'] = data_hr['buy_price']
        data['buy_signal'] = data_hr['buy_signal']
        data['open_adj'] = data_hr['open_adj']
        return data    
    
    def input_sell_signal_daily(self, data): # sell_signal, close_adj
        data_hr = data.copy()
        end_time = data_hr.index[0].hour
        data_hr['sell_signal'] = [1 if date.hour==9 else np.nan for date in data_hr.index]
        data_hr['sell_signal'] = data_hr['sell_signal'].shift(-1)
        data_hr['close_adj'] = np.where(data_hr['sell_signal']==1, data_hr['close'], np.nan) #원래는 sell_signal==1, sell_price
        if 'sell_signal' in data.columns:
            data['sell_signal'] = np.where(data_hr['sell_signal']==1, data_hr['sell_signal'], data['sell_signal'])
            data['close_adj'] = np.where(ata_hr['sell_signal']==1, data_hr['close_adj'], data['close_adj'])
        else:
            data['sell_signal'] = data_hr['sell_signal']
            data['close_adj'] = data_hr['close_adj']
        return data

    def input_sell_signal_stoploss(self, data, stoploss_ratio=0.06):
        # 순서를 바꿔서 copy한 뒤에 data_hr에 np.nan을 넣으면 나중에 sell_signal, close_adj가 입력안되는 문제 발생.
        if 'sell_signal' not in data.columns:
            data['sell_signal'] = np.nan
            data['close_adj'] = np.nan
        data_hr = data.copy()
#         stoploss_ratio =0.06
        status = 0
        list_sell_price = []
        for i in range(len(data_hr)):
            data_time = data_hr.iloc[i]
            if status == 0:
                list_sell_price.append(np.nan)
                if data_time['buy_signal'] == 1:
                    status = 1
                    max_price = data_time['high']
            else:
                max_price = max( [data_time['high'], max_price] )
                sell_price = max_price * (1 - stoploss_ratio)
                list_sell_price.append(sell_price)
                if data_time['low'] < sell_price:
                    data_time['sell_signal'] = 1
                    data_time['close_adj'] = sell_price
                if data_time['sell_signal'] == 1:
                    status = 0
                    max_price = 0
        data['sell_price'] = list_sell_price
        data['sell_signal'] = data_hr['sell_signal']
        data['close_adj'] = data_hr['close_adj']
        return data

    def input_sell_signal_CE(self, data, rolling_timeframe=22, multiplier=3):
        # 순서를 바꿔서 copy한 뒤에 data_hr에 np.nan을 넣으면 나중에 sell_signal, close_adj가 입력안되는 문제 발생.
        if 'sell_signal' not in data.columns:
            data['sell_signal'] = np.nan
            data['close_adj'] = np.nan
        data_hr = data.copy()
        data_hr['atr'] = cal_ma(data_hr['high'].rolling(24*6).max() - data_hr['low'].rolling(24*6).min(), method='sma', length=rolling_timeframe*24*6)
        data_hr['rolling_max'] = data_hr['high'].rolling(rolling_timeframe*24*6).max()
        data_hr['close_adj'] = data_hr['rolling_max'].shift(1)-data_hr['atr'].shift(1)*multiplier
        data_hr['sell_signal'] = np.where(data_hr['low']<data_hr['close_adj'], 1, np.nan)
        data['sell_signal'] = data_hr['sell_signal']
        data['close_adj'] = data_hr['close_adj']
        return data

    def input_buy_signal_mr(self, data, rolling_timeframe=2, multiplier=2):
        data_hr = data.copy()
        data_hr['atr'] =  cal_ma(data_hr['high'].rolling(6).max() - data_hr['low'].rolling(6).min(), method='sma', length=rolling_timeframe*24*6)
        data_hr['rolling_mean'] = data_hr['close'].rolling(rolling_timeframe*24*6).mean()

        data_hr['buy_price'] = data_hr['rolling_mean'].shift(1) - data_hr['atr'].shift(1)*multiplier
       # signal 넣기
        data_hr['buy_signal'] = np.where((data_hr['low'] < data_hr['buy_price'])&(data_hr['open']>data_hr['buy_price']), 1, np.nan)
        # 수익률적용위해 매수시점의 수익률은 open_adj로 입력 -> return계산할때 넣는걸로 변경
        data_hr['open_adj'] = np.where(data_hr['buy_signal']==1, data_hr['buy_price'], np.nan)
        data['buy_price'] = data_hr['buy_price']
        data['buy_signal'] = data_hr['buy_signal']
        data['open_adj'] = data_hr['open_adj']
        return data
    
    def input_sell_signal_mr(self, data, rolling_timeframe=2, multiplier=3):
        # 순서를 바꿔서 copy한 뒤에 data_hr에 np.nan을 넣으면 나중에 sell_signal, close_adj가 입력안되는 문제 발생.
        if 'sell_signal' not in data.columns:
            data['sell_signal'] = np.nan
            data['close_adj'] = np.nan
        data_hr = data.copy()
#        data_hr['atr'] = cal_ma(data_hr['high'].rolling(24*6).max() - data_hr['low'].rolling(24*6).min(), method='sma', length=rolling_timeframe*24*6)
        data_hr['rolling_mean'] = data_hr['close'].rolling(rolling_timeframe*24*6).mean()
        data_hr['close_adj'] = data_hr['rolling_mean'].shift(1)
        data_hr['sell_signal'] = np.where((data_hr['high'] > data_hr['close_adj'])&(data_hr['open']<data_hr['close_adj']), 1, np.nan)
        data['sell_signal'] = data_hr['sell_signal']
        data['close_adj'] = data_hr['close_adj']
        return data

    def input_signal_MA(self, data, l_fast=2, l_slow=8, forecast=False):
        data_hr = data.copy()
        data_hr['close_fast'] = cal_ma(data_hr['close'] , method='ema', length=int(l_fast)*6*24)
        data_hr['close_slow'] = cal_ma(data_hr['close'] , method='ema', length=int(l_slow)*6*24)

        data_hr['crossover'] = data_hr['close_fast'] - data_hr['close_slow']
        data['crossover'] = data_hr['crossover']
        if forecast == 0:
            data_hr['status_fill'] = np.where(data_hr['crossover'].shift(1)>0, 1, 0)
        else:
            data_hr['price_std'] = data_hr['close'].std() * ((6*24)**0.5)
            data_hr['crossover_std'] = data_hr['crossover'] / data_hr['price_std']
            data_hr['forecast_scalar'] = 10
            data_hr['forecast'] = data_hr['crossover_std'] * data_hr['forecast_scalar']
            data['forecast'] = data_hr['forecast']
#             data_hr['status_fill'] = np.where(data_hr['forecast'].shift(1)>=20, 1, np.where(data_hr['forecast'].shift(1)<=0, 0, data_hr['forecast'].shift(1)/20) )
            data_hr['status_fill'] = np.where(data_hr['forecast'].shift(1)>=20, 1, np.where(data_hr['forecast'].shift(1)>=10, 0.75, np.where(data_hr['forecast'].shift(1)>0, 0.5, 0) ) )
        data['open_adj'] = data['open']
        data['close_adj'] = data['close']
        data['status_fill'] = data_hr['status_fill']  # 0~1
        data['status_fill'].fillna(0,inplace=True)
        return data
    
    def output_status(self, data):
        data_hr = data.copy()
        if not 'status_fill' in data_hr.columns:
            data_hr['status'] = np.where(data_hr['sell_signal'].shift(1)==1, 0, np.where(data_hr['buy_signal']==1, 1, np.nan))
            data_hr['status_fill'] = data_hr['status'].fillna(method='ffill')
            data_hr['status_fill'].fillna(0,inplace=True)
        # 매도 바로 다음 시점 배수가 붙으면 구분이 안되는 단점.
        return data_hr
    
    # data_raw를 일별 return으로 변환하기.
    def cal_return(self, data, trading_fee):
        data_hr = data
        data_hr['open_adj'] = np.where((data_hr.status_fill==1)&(data_hr.status_fill.shift(1)==0),data_hr['open_adj'],data_hr['open'])
        data_hr['close_adj'] = np.where((data_hr.status_fill==1)&(data_hr.status_fill.shift(-1)==0),data_hr['close_adj'],data_hr['close'])
#         data_hr['status_fee'] = np.where(((data_hr.status_fill==1)&(data_hr.status_fill.shift(1)==0))|((data_hr.status_fill==1)&(data_hr.status_fill.shift(-1)==0)), 1, 0)  # 0->1이면1에, 1->0이면 1에
        # status_fill 이 binary하지 않게 변할 때 fee를 반영할 수 있도록.
        data_hr['status_fee'] = 0
        data_hr['status_fee'] = data_hr['status_fee'] + np.where(data_hr.status_fill > data_hr.status_fill.shift(1), data_hr.status_fill - data_hr.status_fill.shift(1), 0)
        data_hr['status_fee'] = data_hr['status_fee'] + np.where(data_hr.status_fill > data_hr.status_fill.shift(-1), data_hr.status_fill - data_hr.status_fill.shift(-1), 0)

        data_hr['return'] = np.log10(data_hr.close / data_hr.open)
        data_hr['strategy_return'] = np.log10(data_hr.close_adj / data_hr.open_adj)*data_hr['status_fill']
        # 제대로 계산되는지 평가
        data_hr['incon'] = np.where(data_hr['close_adj']>data_hr['high'], 1, 0)
        if data_hr['incon'].sum() != 0:
#             print(f"{self.strategy_list}\n{data_hr[data_hr['incon']==1]}")
            print(f"{self.strategy_list}__{data_hr['incon'].sum()}")
#         data_hr['status'] = np.where(data_hr['status'].notna(), 1, 0)
        # 수수료 반영
        if not self.trading_fee == 0:
            data_hr['strategy_return'] = data_hr['strategy_return'] + data_hr['status_fee'] * np.log10(1-(trading_fee*0.01))
        data_d = data_hr[['strategy_return']].resample('24H', origin=data_hr.index[0].replace(hour=9)).apply({'strategy_return':'sum'})
#         if (data_d.buy_entry.sum() == 0) & (data_d.buy_exit.sum() == 0):
#             raise ValueError('Buy_Entry, Buy_Exit 가 없음')
        return data_hr, data_d['strategy_return']      


#         dict_ohlcv = {'return':'sum','strategy_return':'sum','status_fee':'sum'}
#         data_d = data_hr[['return','strategy_return','status_fee']].resample('24H', origin=data_hr.index[0].replace(hour=9)).apply(dict_ohlcv)
#         buy_entry = np.where(data_d.strategy_return == 0, 0, 1)
# #         data_d['buy_entry'] = buy_entry
# #         data_d['buy_exit'] = buy_entry
# #         if (data_d.buy_entry.sum() == 0) & (data_d.buy_exit.sum() == 0):
# #             raise ValueError('Buy_Entry, Buy_Exit 가 없음')
#         # 수수료 반영
#         if not self.trading_fee == 0:
#             data_d['strategy_return'] = data_d['strategy_return'] + data_d['status_fee'] * np.log10(1-(trading_fee*0.01))
#         return data_d['strategy_return']      
        
    def calculate(self):
        data_raw = self.data_hr
        for strategy in self.strategy_list:
            if 'buy' in strategy:
                if 'volbreak_d' in strategy:
                    data_raw = self.input_buy_signal_vol_d(data_raw)
                elif 'volbreak_m' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_buy_signal_vol_m(data_raw)
                    else:
                        _, _, volbreak_daily = strategy.split(', ')
                        data_raw = self.input_buy_signal_vol_m(data_raw, volbreak_daily=float(volbreak_daily))
                elif 'supertrend' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_buy_signal_supertrend(data_raw)
                    else:
                        _, _, timeframe, multiplier = strategy.split(', ')
                        data_raw = self.input_buy_signal_supertrend(data_raw, rolling_timeframe=int(timeframe), multiplier=float(multiplier))
                elif 'mr' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_buy_signal_mr(data_raw)
                    else:
                        _, _, timeframe, multiplier = strategy.split(', ')
                        data_raw = self.input_buy_signal_mr(data_raw, rolling_timeframe=int(timeframe), multiplier=float(multiplier))
                else:
                    raise ValueError('buy전략 이름 확인 필요')
            elif 'sell' in strategy:
                if 'daily' in strategy:
                    data_raw = self.input_sell_signal_daily(data_raw)
                elif 'stoploss' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_sell_signal_stoploss(data_raw)
                    else:
                        _, _, stoploss_ratio = strategy.split(', ')
                        data_raw = self.input_sell_signal_stoploss(data_raw, stoploss_ratio=float(stoploss_ratio))
                elif 'CE' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_sell_signal_CE(data_raw)
                    else:
                        _, _, timeframe, multiplier = strategy.split(', ')
                        data_raw = self.input_sell_signal_CE(data_raw, rolling_timeframe=int(timeframe), multiplier=float(multiplier))
                elif 'mr' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_sell_signal_mr(data_raw)
                    else:
                        _, _, timeframe, multiplier = strategy.split(', ')
                        data_raw = self.input_sell_signal_mr(data_raw, rolling_timeframe=int(timeframe), multiplier=float(multiplier))
                else:
                    raise ValueError('sell전략 이름 확인 필요')
            elif 'total' in strategy:
                if 'ma' in strategy:
                    if len(strategy.split(', ')) == 2:
                        data_raw = self.input_signal_MA(data_raw)
                    else:
                        _, _, l_fast, l_slow, forecast = strategy.split(', ')
                        data_raw = self.input_signal_MA(data_raw, l_fast=l_fast,l_slow=l_slow, forecast=int(forecast))

        data_raw = self.output_status(data_raw)
        self.data_hr = data_raw
        data_d = self.cal_return(data_raw,self.trading_fee)
        return data_d

In [34]:
data_hr.describe()

,open,high,low,close,volume,value,close_fast,close_slow,crossover,price_std,crossover_std,forecast_scalar,forecast,status_fill
count,2.460140e+05,2.460150e+05,2.460150e+05,2.460150e+05,246015.000000,2.460150e+05,2.457280e+05,2.448640e+05,2.448640e+05,2.460150e+05,244864.000000,246015.0,244864.000000,244863.000000
mean,2.451688e+07,2.456692e+07,2.446714e+07,2.451689e+07,61.585614,1.578355e+09,2.451794e+07,2.451838e+07,1.479481e+04,2.565443e+08,0.000058,10.0,0.000577,0.000688
std,2.137873e+07,2.142155e+07,2.133647e+07,2.137869e+07,94.137294,2.945040e+09,2.137475e+07,2.135467e+07,1.201333e+06,3.356046e-04,0.004683,0.0,0.046827,0.001514
min,3.574000e+06,3.576000e+06,3.562000e+06,3.574000e+06,0.000046,4.999273e+02,3.654823e+06,3.787033e+06,-7.659547e+06,2.565443e+08,-0.029857,10.0,-0.298566,0.000000
25%,8.556250e+06,8.573000e+06,8.540000e+06,8.556500e+06,16.693916,1.833022e+08,8.559370e+06,8.569780e+06,-2.953528e+05,2.565443e+08,-0.001151,10.0,-0.011513,0.000000
50%,1.231600e+07,1.233600e+07,1.229600e+07,1.231600e+07,33.411880,5.372677e+08,1.231425e+07,1.230559e+07,3.335015e+03,2.565443e+08,0.000013,10.0,0.000130,0.000007
75%,4.115925e+07,4.125150e+07,4.103650e+07,4.115850e+07,67.454143,1.781421e+09,4.097191e+07,4.128439e+07,3.087527e+05,2.565443e+08,0.001204,10.0,0.012035,0.000602
max,8.269900e+07,8.270000e+07,8.225600e+07,8.269900e+07,3559.807130,1.251281e+11,8.045239e+07,7.852588e+07,5.936097e+06,2.565443e+08,0.023139,10.0,0.231387,0.011569


In [9]:
data_all = pd.read_pickle('./data/data_coin_minute10.pkl')
data_all

,open,high,low,close,volume,value,date,coin
0,4201000.0,4241000.0,4195000.0,4227000.0,15.940028,6.722878e+07,2017-09-25 12:00:00,KRW-BTC
1,4215000.0,4244000.0,4210000.0,4227000.0,16.224769,6.854317e+07,2017-09-25 12:10:00,KRW-BTC
2,4215000.0,4242000.0,4191000.0,4221000.0,16.144995,6.806705e+07,2017-09-25 12:20:00,KRW-BTC
3,4219000.0,4242000.0,4202000.0,4242000.0,17.943481,7.572972e+07,2017-09-25 12:30:00,KRW-BTC
4,4227000.0,4233000.0,4199000.0,4224000.0,19.122032,8.065820e+07,2017-09-25 12:40:00,KRW-BTC
...,...,...,...,...,...,...,...,...
535271,2289000.0,2292000.0,2284000.0,2285000.0,338.979249,7.752019e+08,2022-11-05 13:50:00,KRW-ETH
535272,2284000.0,2291000.0,2284000.0,2288000.0,267.763797,6.125802e+08,2022-11-05 14:00:00,KRW-ETH
535273,2288000.0,2290000.0,2286000.0,2289000.0,195.070948,4.463511e+08,2022-11-05 14:10:00,KRW-ETH
535274,2289000.0,2291000.0,2288000.0,2289000.0,85.289751,1.952890e+08,2022-11-05 14:20:00,KRW-ETH


In [30]:
data_hr = case1.strategy.data_hr.iloc[:,:6]
l_fast = 2
l_slow = 8
data_hr['close_fast'] = cal_ma(data_hr['close'] , method='ema', length=int(l_fast)*6*24)
data_hr['close_slow'] = cal_ma(data_hr['close'] , method='ema', length=int(l_slow)*6*24)
data_hr['crossover'] = data_hr['close_fast'] - data_hr['close_slow']
data_hr['price_std'] = data_hr['close'].std() * ((6*24)**0.5)
data_hr['crossover_std'] = data_hr['crossover'] / data_hr['price_std']
data_hr['forecast_scalar'] = 10
data_hr['forecast'] = data_hr['crossover_std'] * data_hr['forecast_scalar']
data_hr['status_fill'] = np.where(data_hr['forecast'].shift(1)>=20, 1, np.where(data_hr['forecast'].shift(1)<=0, 0, data_hr['forecast'].shift(1)/20) )


In [7]:
date_start = datetime(2018,1,1,9)
# date_start = datetime(2022,1,1,9)
# date_start = datetime(2022,11,1,9)
date_end = datetime(2022,8,31,9)
# date_end = datetime(2022,11,4,9)
coin_code = 'KRW-BTC'
case1 = Backtest(date_start,date_end,coin_code)
case1.trading_fee = 0.1
# case1.cal_strategy(f'mr', Strategy_Volatility_new,[f'buy, volbreak_m',f'sell, CE'])
case1.cal_strategy(f'ma0', Strategy_Volatility_new,[f'total, ma'])
case1.cal_strategy(f'ma1', Strategy_Volatility_new,[f'total, ma, 2, 8, 0'])
case1.cal_strategy(f'ma2', Strategy_Volatility_new,[f'total, ma, 2, 8, 1'])

case1.analyze_result(add=False)

,name,start,end,times_in_market,avg_rtn,total_profit,cagr,mdd,sortino,worst_rtn,RoR
0,return,180101,220831,100.0,0.09 / 2.34 / -2.37,42.05,7.814,-86.81,0.54,"[-40.72, -36.26, -35.59, -35.1, -34.87]",0.280
1,return,180101,211231,100.0,0.14 / 2.42 / -2.39,194.52,31.003,-86.81,0.88,"[-40.72, -36.26, -35.59, -35.1, -24.82]",0.257
2,return,220101,220831,100.0,-0.25 / 1.81 / -2.27,-51.77,-66.705,-56.67,-1.74,"[-34.87, -18.99, -16.76, -10.96, -10.94]",0.433
3,ma0,180101,220831,57.0,0.29 / 2.32 / -1.95,878.88,63.058,-42.75,1.74,"[-15.91, -13.68, -13.32, -11.83, -11.0]",0.140
4,ma0,180101,211231,59.0,0.35 / 2.41 / -1.98,1203.35,90.005,-41.34,2.16,"[-15.91, -13.68, -13.32, -11.0, -10.98]",0.121
5,ma0,220101,220831,46.0,-0.23 / 1.53 / -1.70,-24.89,-35.065,-34.88,-1.41,"[-11.83, -7.82, -5.73, -5.72, -3.27]",0.186
6,ma1,180101,220831,57.0,0.29 / 2.32 / -1.95,878.88,63.058,-42.75,1.74,"[-15.91, -13.68, -13.32, -11.83, -11.0]",0.122
7,ma1,180101,211231,59.0,0.35 / 2.41 / -1.98,1203.35,90.005,-41.34,2.16,"[-15.91, -13.68, -13.32, -11.0, -10.98]",0.131
8,ma1,220101,220831,46.0,-0.23 / 1.53 / -1.70,-24.89,-35.065,-34.88,-1.41,"[-11.83, -7.82, -5.73, -5.72, -3.27]",0.169
9,ma2,180101,220831,57.0,0.13 / 1.15 / -0.98,212.87,27.694,-24.34,1.56,"[-8.3, -7.09, -6.9, -6.1, -5.66]",0.023


In [38]:
case1.data_dict['ma1'].to_csv('./result_ma1.csv')

In [20]:
# arr = case1.data_d['ma1']
est_list = np.array([np.power(10, (np.random.choice(arr, 30)).sum()) for _ in range(100)]) 
sum(est_list<0.9)

11

In [18]:
def _analyze_result_ind(data):
    data['status_fill_temp'] = np.where((data.status_fill==1) & (data.status_fill.shift(1)==0), 1, 0).cumsum()
    data['status_fill_temp'] = data['status_fill'] * data['status_fill_temp']
    rslt = data[data.status_fill_temp != 0].groupby('status_fill_temp')['strategy_return'].agg(['count','sum'])
#    return rslt
    array_trading_period = rslt['count']
    array_trading_profit = np.power(10,rslt['sum'])
    number_of_trading = len(array_trading_period)
    array_trading_profit_positive = array_trading_profit[array_trading_profit>1]
    array_trading_profit_negative = array_trading_profit[array_trading_profit<=1]
    number_of_trading_positive = len(array_trading_profit_positive)
    number_of_trading_negative = len(array_trading_profit_negative)
    hit_ratio = number_of_trading_positive / number_of_trading
    expactacy_per_trade = (array_trading_profit_positive.sum()+array_trading_profit_negative.sum())/number_of_trading-1
    average_gain = array_trading_profit_positive.mean()-1
    average_loss = array_trading_profit_negative.mean()-1
    largest_gain = array_trading_profit_positive.max()-1
    largest_loss = array_trading_profit_negative.min()-1
    analysis_ind = [number_of_trading , number_of_trading_positive, round(hit_ratio,2), round(-average_gain/average_loss,2), round(expactacy_per_trade*100,2), round(average_gain*100,2), round(average_loss*100,2)]
    return analysis_ind

In [19]:
aa = case1.data_dict['ma']
np.where((aa.status_fill==1) & (aa.status_fill.shift(1)==0), 1, 0)

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
_analyze_result_ind(case1.data_dict['ma'])

[17, 4, 0.24, 1.38, -1.58, 4.95, -3.59]

In [4]:
date_start = datetime(2018,1,1,9)
# date_start = datetime(2022,1,1,9)
# date_start = datetime(2022,11,1,9)
# date_end = datetime(2022,8,31,9)
date_end = datetime(2022,11,4,9)
coin_code = 'KRW-BTC'
case1 = Backtest(date_start,date_end,coin_code)
case1.trading_fee = 0.1
# case1.cal_strategy('base', Strategy_Volatility_new,['base'])
# for i in [0.1, 0.2, 0.3]:
#     for j in [15,22,30]:
#         for k in [2,3,4]:
# i = 0.15
# j = 22
# k = 3

case1.cal_strategy(f'base', Strategy_Volatility_new,[f'buy, volbreak_d',f'sell, daily'])
case1.cal_strategy(f'base_SL0.05', Strategy_Volatility_new,[f'buy, volbreak_d',f'sell, stoploss, 0.05'])
case1.cal_strategy(f'base_SL0.1', Strategy_Volatility_new,[f'buy, volbreak_d',f'sell, stoploss, 0.1'])
for i in [0.05, 0.1, 0.2, 0.3]:
    case1.cal_strategy(f'Con{i}_SL0.05', Strategy_Volatility_new,[f'buy, volbreak_m, {i}',f'sell, stoploss, 0.05'])
    case1.cal_strategy(f'Con{i}_SL0.1', Strategy_Volatility_new,[f'buy, volbreak_m, {i}',f'sell, stoploss, 0.1'])
for i in [0.05, 0.1, 0.2, 0.3]:
    for j in [15,22,30]:
        for k in [2,3,4]:
            case1.cal_strategy(f'Con{i}_CE{j}/{k}', Strategy_Volatility_new,[f'buy, volbreak_m, {i}',f'sell, CE, {j}, {k}'])
# case1.cal_strategy(f'{i}_{j}_{k}', Strategy_Volatility_new,[f'buy, volbreak_m, {i}',f'sell, CE, {j}, {k}'])
# case1.cal_strategy(f'name1', Strategy_Volatility_new,[f'buy, volbreak_m, {i}',f'sell, stoploss'])

case1.analyze_result()

KeyboardInterrupt: 

In [ ]:
import quantstats as qs
qs.reports.basic(_power10(case1.data_d['mr'])-1,benchmark=(_power10(case1.data_d['return'])-1))

# ML

In [25]:
# variable 생성 함수 공통으로 사용.

def make_ml_dataset(data_raw):
    data_raw['target'] = np.where(data_raw['return'] > np.log10(1), 1, 0)

    data_raw['rtn_l1'] = data_raw['return'].rolling(5).sum().shift(1)
    data_raw['rtn_l2'] = data_raw['return'].rolling(20).sum().shift(1)
    data_raw['rtn_l3'] = data_raw['return'].rolling(60).sum().shift(1)

    data_raw['u_noise'] = 1- np.abs(data_raw['high']-data_raw['close'])/(data_raw['high']-data_raw['low'])
    data_raw['d_noise'] = 1- np.abs(data_raw['close']-data_raw['low'])/(data_raw['high']-data_raw['low'])

    data_raw['u_noise_l1'] = data_raw['u_noise'].rolling(5).mean().shift(1)
    data_raw['d_noise_l1'] = data_raw['d_noise'].rolling(5).mean().shift(1)

    data_raw['value_cng'] = data_raw['value'].pct_change()
    data_raw['value_cng_l1'] = data_raw['value_cng'].rolling(5).mean().shift(1)

    data_raw.dropna(inplace=True)
    ml_dataset = data_raw[['rtn_l1', 'rtn_l2', 'rtn_l3', 'u_noise_l1', 'd_noise_l1','value_cng_l1']].to_numpy()
    target = data_raw['target'].to_numpy()
    return ml_dataset, target

In [27]:
model = joblib.load('./model/model2.pkl')
# score = loaded_model.score(X,y)

def calculate_ml(data_raw, model):
    start_hour = 9

    data_raw_hourly_temp = data_raw.copy()
    data_raw_hourly_temp.index.name = None
    start_datetime = data_raw_hourly_temp.index[0].replace(hour=start_hour)
    data_raw_hourly_temp = data_raw_hourly_temp.loc[start_datetime:]
    # 일별 자료로 변환한 후 target_price계산.
    dict_ohlcv = {'open':'first','high':'max','low':'min','close':'last','value':'sum','return':'sum'}
    data_raw_temp = data_raw_hourly_temp.resample('24H', origin=start_datetime ).apply(dict_ohlcv).loc[start_datetime:]

    input_data, _ = make_ml_dataset(data_raw_temp)
    buy_entry = model.predict(input_data)

    data_raw_temp = data_raw_temp.iloc[-len(buy_entry):]

    data_raw_temp['buy_entry'] = buy_entry
    data_raw_temp['buy_exit'] = buy_entry

    data_raw_temp['strategy_return'] = np.where(data_raw_temp.buy_entry == 1, data_raw_temp['return'], 0)

    data_raw = data_raw_temp[['return','strategy_return','buy_entry','buy_exit']]
    return data_raw
#     return input_data

In [7]:
date_start = datetime(2017,10,1)
# date_start = datetime(2022,1,1)
date_end = datetime(2022,8,31)
coin_code = 'KRW-BTC'
data_raw = define_data(date_start-relativedelta(days=10),date_end,coin_code,time_start=9,timeframe='hourly')
# case1 = Backtest(data_raw, Strategy_Volatility, strategy_list=['ml'])


In [71]:
case1.trading_fee = 0.1
case1.cal_strategy()
case1.test_result = case1.test_result.loc[date_start:date_end]
case1.cal_result()
print(case1.analyze_result_cont())
print(case1.analyze_result_ind())

전체수익 : 54.01 %, CAGR : 128.24 %, MDD : -18.71 %, STD : 14.43 %, Sharpe : 8.89
거래횟수 (수익, 승률 ) : 85 ( 54, 0.64), 평균 수익률 % (수익, 손실, 손익비) : 0.54 (1.92, -1.86, 1.03)


## 상관관계

In [ ]:
data=pd.DataFrame()
data['return'] = np.log10(data_raw['close']/data_raw['open'])
data['l1'] = data['return'].shift(1)
data['l4'] = data['return'].rolling(4).mean().shift(1)
data['l12'] = data['return'].rolling(12).mean().shift(1)
data['l24'] = data['return'].rolling(24).mean().shift(1)
data['l48'] = data['return'].rolling(48).mean().shift(1)
data['r1'] = data['return']
data['r4'] = data['return'].rolling(4).mean().shift(-3)
data['r12'] = data['return'].rolling(12).mean().shift(-11)
data['r24'] = data['return'].rolling(24).mean().shift(-23)
data['r48'] = data['return'].rolling(48).mean().shift(-47)
data.corr()

# TEST

In [100]:
rtn_list = []
rtn_list_1m = []
rtn_list_2m = []
rtn_list_3m = []
for col_name in case1.data_d.columns[12:]:
    data_each = case1.data_d[col_name]
    rtn_record = qs.stats.monthly_returns( _power10(data_each)-1)
    rtn_array = pd.Series(rtn_record.iloc[:,:12].values.flatten())
    rtn_list += list(rtn_array)
    rtn_list_1m += [np.nan]
    rtn_list_1m += list(rtn_array[:-1])
    rtn_list_2m += [np.nan]
    rtn_list_2m += list(rtn_array.rolling(2).mean()[:-1])
    rtn_list_3m += [np.nan]
    rtn_list_3m += list(rtn_array.rolling(3).mean()[:-1])

rtn_df = pd.DataFrame([rtn_list, rtn_list_1m, rtn_list_2m, rtn_list_3m]).T
rtn_df.dropna(inplace=True)
rtn_df.corr()



,0,1,2,3
0,1.000000,0.134429,0.098919,0.111479
1,0.134429,1.000000,0.750407,0.608919
2,0.098919,0.750407,1.000000,0.846906
3,0.111479,0.608919,0.846906,1.000000


# 과거 code 저장

In [100]:
class Backtest():
    def __init__(self, price_data: pd.DataFrame, strategy_cls:type, strategy_list:None):
        self.price_data = price_data
        self.strategy_list = strategy_list
        self.strategy : Strategy = strategy_cls(self.price_data,self.strategy_list)
        
        self.trading_fee = 0.05
        self.case_name = None
        
    def cal_strategy(self):
        self.test_result = self.strategy.calculate()  # finish_daily : 일별로 거래 종료되어 position이 거래횟수
    def cal_result(self):
        data = self.test_result.copy()
        # 에러발생코드
#         data['strategy_return'].fillna(0,inplace=True)
        if (data.buy_entry.sum() == 0) & (data.buy_exit.sum() == 0):
            raise ValueError('Buy_Entry, Buy_Exit 가 없음')
        # 수수료 반영
        if not self.trading_fee == 0:
            data['strategy_return'] = data['strategy_return'] + data['buy_entry'] * np.log10(1-(self.trading_fee*0.01))
            data['strategy_return'] = data['strategy_return'] + data['buy_exit'] * np.log10(1-(self.trading_fee*0.01))
        
        data['strategy'] = data['strategy_return'].fillna(0)      
        data['creturns'] = data['return'].cumsum()
        data['cstrategy'] = data['strategy'].cumsum()
#         aperf = data['cstrategy'].iloc[-1]
#         operf = aperf - data['creturns'].iloc[-1]
        self.results = data
#         return round(aperf, 2), round(operf, 2)
        return round(data['creturns'].iloc[-1],2), round(data['cstrategy'].iloc[-1],2)
    def analyze_result_ind(self):
        data = self.results.copy()
        self.analysis_ind = _analyze_result_ind(data)
        return f'거래횟수 (수익, 승률 ) : {self.analysis_ind[0]} ( {self.analysis_ind[1]}, {self.analysis_ind[2]:.2f}), 평균 수익률 % (수익, 손실, 손익비) : {self.analysis_ind[4]:.2f} ({self.analysis_ind[5]:.2f}, {self.analysis_ind[6]:.2f}, {self.analysis_ind[3]:.2f})'
#     self.analysis_ind
# [number_of_trading , number_of_trading_positive, hit_ratio, -average_gain/average_loss, expactacy_per_trade, average_gain, average_loss]
    def plot_results(self, markup=False):
        data = self.results.copy()
        _plot_results(data, markup)
    def analyze_result_per(self):
        data = self.results.copy()
        self.analysis_per = _analyze_result_per(data)
        return self.analysis_per
    def plot_analysis_per(self):
        data = self.analysis_per.copy()
        _plot_analysis_per(data)
    
    def analyze_result_cont(self):
        data = self.results.copy()
        self.analysis_cont = _analyze_result_cont(data)
        return f'전체수익 : {self.analysis_cont[0]:.2f} %, CAGR : {self.analysis_cont[1]:.2f} %, MDD : {self.analysis_cont[2]:.2f} %, STD : {self.analysis_cont[3]:.2f} %, Sharpe : {self.analysis_cont[4]:.2f}'
    def add_log_result(self):
        _add_log_result([self.case_name], self.analysis_ind, self.analysis_cont)
    def delete_log_result(self):
        _delete_log_result()

In [95]:
# rollover를 나중에 더하는 함수로 변경.
# def calculate_rollover_logic(self):
#     data_raw_temp = self.data_raw.copy()
#     rollover_signal = (data_raw_temp.high - data_raw_temp.close)/(data_raw_temp.high - data_raw_temp.low) < 0.1
#     trade_start_signal = ( np.isnan(data_raw_temp.open_AT) == False ) & ( np.isnan(data_raw_temp.open_AT).shift(1) == True )
#     trade_on = np.where(rollover_signal==False,False,np.where(trade_start_signal,True,np.nan))
#     data_raw_temp['rollover_trade_on'] = trade_on
#     data_raw_temp['rollover_trade_on'].fillna(method='ffill', inplace=True)
#     data_raw_temp['strategy_return'] = np.where(trade_on==False, 0, np.where(trade_start_signal, np.log10(data_raw_temp.close/data_raw_temp.open_AT), np.log10(data_raw_temp.close/data_raw_temp.open) ) )
#     # buy_trade_entry = (data_raw_temp.stategy_return != 0) & (data_raw_temp.stategy_return.shift(1) == 0)
#     buy_trade_exit = (data_raw_temp.stategy_return != 0) & (data_raw_temp.stategy_return.shift(-1) == 0)
#     data_raw_temp['buy_entry'] =  trade_start_signal
#     data_raw_temp['buy_exit'] = buy_trade_exit
#     self.data_raw = data_raw_temp

#  기본 (k, range의 변형 가능, stoploss가 쓰이면 _AT로 계산) -> filter ( np.where(filter_condition, df.buy_entry, 0) ) -> stoploss or rollover

class Strategy_Volatility(Strategy):
    def __init__(self,data_raw, strategy_list=['base']):
        self.data_raw = data_raw.reset_index(drop=True)
        self.strategy_list = strategy_list
        self.gap_second = (self.data_raw.date[1]-self.data_raw.date[0]).total_seconds()
        
        if int(self.gap_second) == 86400:
            # 코인은 일 가격의 date정보가 9시로 되어있기때문에, 일정보까지만 저장하게 변환.
            self.data_raw.index = self.data_raw.date
            self.data_raw_for_hour = self.data_raw.copy()
            self.data_raw = self.data_raw.resample('D').mean()  # 일 단위 값으로 index를 일까지 표현되게 수정.
        else:
            self.data_raw.index = self.data_raw.date
        
#         self.stoploss = 0.05
#         self.k_value = 0.5

# 4주 고가에 돌파 시 매수. 2주 저가 붕괴 시 매도.
    def calculate_turtle(self):
        data_raw_temp = self.data_raw.copy()
        
        data_raw_temp['max_days'] = data_raw_temp.close.rolling(20).max()
        data_raw_temp['min_days'] = data_raw_temp.close.rolling(10).min()

        # 하나의 numpy에 high>max_days 면 1, low < min_days 면 0, 나머지는 nan. -> fillna위에값 -> 0 에서 1로 바뀌면 매수buy_entry, 1에서 0으로 바뀌면 매도buy_exit, shift활용)
        trade_status = np.where( data_raw_temp['high'] >= data_raw_temp['max_days'], 1, np.where( data_raw_temp['low'] <= data_raw_temp['min_days'], 0, np.nan ) ) 
        data_raw_temp['trade_status'] = list(trade_status)
    #         data_raw_temp['trade_status'] = data_raw_temp['trade_status'].shift(1)
        data_raw_temp['trade_status'].fillna(method='ffill', inplace=True)
        data_raw_temp['strategy_return'] = data_raw_temp['return'] * data_raw_temp['trade_status'].shift(1)
    #         trade_status
        data_raw_temp['buy_entry'] = np.where( (data_raw_temp.trade_status==1) & (data_raw_temp.trade_status.shift(1) == 0), 1, 0)
        data_raw_temp['buy_exit'] = np.where( (data_raw_temp.trade_status==0) & (data_raw_temp.trade_status.shift(1)) == 1, 1, 0)
        self.data_raw = data_raw_temp.copy()

    def calculate_volatility_breakout(self, target_vol = None):
        data_raw_temp = self.data_raw.copy()
        
#         price_range = cal_ma(data_raw_temp['high']-data_raw_temp['low'],method='ema',length=10)
        price_range = data_raw_temp['high']-data_raw_temp['low']
        
        # k_value를 dynamic하게 적용 가능.
#         k_value = np.maximum( 0.5, ( abs( data_raw_temp.open.shift(1) - data_raw_temp.close.shift(1) ) / price_range.shift(1) ))
#         target_price = data_raw_temp['open'] + price_range.shift(1) * k_value
        
        k_value = 0.5
        target_price = data_raw_temp['open'] + price_range.shift(1) * k_value
        data_raw_temp['price_range'] = price_range
        data_raw_temp['target_price'] = target_price
        
        return_adjust = np.where(target_price < data_raw_temp['high'], np.log10(data_raw_temp.close / target_price), 0)
        
        if target_vol == None:
            data_raw_temp['strategy_return'] = return_adjust
        else:
            trading_portion = np.minimum(1, target_vol/(price_range/data_raw_temp['open']))
            data_raw_temp['strategy_return'] = return_adjust + np.log10(trading_portion.shift(1) + (1-trading_portion.shift(1))/_power10(return_adjust) )

        # buy_condition 적용
        buy_condition = (return_adjust != 0)        

        data_raw_temp['buy_entry'] = buy_condition
        data_raw_temp['buy_exit'] = buy_condition
        self.data_raw = data_raw_temp

    
# score = loaded_model.score(X,y)

    def calculate_ml(self):
        start_hour = 9
        model = joblib.load('./model/model4.pkl')
        data_raw_hourly_temp = self.data_raw.copy()
        data_raw_hourly_temp.index.name = None
        start_datetime = data_raw_hourly_temp.index[0].replace(hour=start_hour)
        data_raw_hourly_temp = data_raw_hourly_temp.loc[start_datetime:]
        # 일별 자료로 변환한 후 target_price계산.
        dict_ohlcv = {'open':'first','high':'max','low':'min','close':'last','value':'sum','return':'sum'}
        data_raw_temp = data_raw_hourly_temp.resample('24H', origin=start_datetime ).apply(dict_ohlcv).loc[start_datetime:]

        input_data, _ = make_ml_dataset(data_raw_temp)
        buy_entry = model.predict(input_data)

        data_raw_temp = data_raw_temp.iloc[-len(buy_entry):]

        data_raw_temp['buy_entry'] = buy_entry
        data_raw_temp['buy_exit'] = buy_entry

        data_raw_temp['strategy_return'] = np.where(data_raw_temp.buy_entry == 1, data_raw_temp['return'], 0)

        self.data_raw = data_raw_temp[['return','strategy_return','buy_entry','buy_exit']]
        return data_raw
        
        
    def calculate_volatility_breakout_stoploss(self, stoploss = 0.02, target_vol=None, start_hour=9):
        data_raw_hourly_temp = self.data_raw.copy()
        data_raw_hourly_temp.index.name = None
#         start_hour = 9
#         stoploss = 'pr'  # pr
        start_datetime = data_raw_hourly_temp.index[0].replace(hour=start_hour)
        data_raw_hourly_temp = data_raw_hourly_temp.loc[start_datetime:]
        # 일별 자료로 변환한 후 target_price계산.
        dict_ohlcv = {'open':'first','high':'max','low':'min','close':'last','value':'sum'}
        data_raw_temp = data_raw_hourly_temp.resample('24H', origin=start_datetime ).apply(dict_ohlcv).loc[start_datetime:]
        price_range = data_raw_temp['high']-data_raw_temp['low']
#         price_range = cal_ma(data_raw_temp['high']-data_raw_temp['low'])
        k_value = np.maximum( 0.5, ( abs( data_raw_temp.open.shift(1) - data_raw_temp.close.shift(1) ) / price_range.shift(1) ))
        k_value = 0.5
        target_price = data_raw_temp['open'] + price_range.shift(1) * k_value
        # range가 작다가 갑자기 상승하는 경우에 휩소에 휘둘리는 것 방지하기 위해.
#         target_price = np.maximum(data_raw_temp['open']*1.02, data_raw_temp['open'] + price_range.shift(1) * k_value)
        data_raw_temp['price_vol'] = price_range.shift(1) * k_value/data_raw_temp['open']  #비율로 계산됨.
        data_raw_temp['price_range'] = price_range
        data_raw_temp['target_price'] = target_price
        data_raw_temp['target_price_low'] = data_raw_temp['open'] - price_range.shift(1) * k_value
        # 변동성이 작은 구간에서 휩소방지위해
#         data_raw_temp['target_price_low'] = np.minimum(data_raw_temp['open']*0.98, data_raw_temp['open'] - price_range.shift(1) * k_value)
        data_raw_temp['return'] = np.log10(data_raw_temp.close / data_raw_temp.close.shift(1))

        # 시간별 자료에 target_price 넣은 후 매수상태 확인.
        data_raw_hourly_temp['price_vol'] = data_raw_temp['price_vol']  #9시에 들어감.
        data_raw_hourly_temp['price_vol'].fillna(method='ffill', inplace=True)
        data_raw_hourly_temp['price_range'] = data_raw_temp['price_range']
        data_raw_hourly_temp['target_price'] = data_raw_temp['target_price']  #9시에 들어감.
        data_raw_hourly_temp['target_price'].fillna(method='ffill', inplace=True)
        data_raw_hourly_temp['target_price_low'] = data_raw_temp['target_price_low']  #9시에 들어감.
        data_raw_hourly_temp['target_price_low'].fillna(method='ffill', inplace=True)
        
        # 일 중에 stoploss로 매도 했어도 target에서 다시 매수할 수 있다.
        data_raw_hourly_temp['status'] = np.where(data_raw_hourly_temp.index.hour == start_hour, 0, 
                                                  np.where((data_raw_hourly_temp.target_price < data_raw_hourly_temp.high)&
                                                           (data_raw_hourly_temp.target_price > data_raw_hourly_temp.low)&
                                                           (data_raw_hourly_temp.target_price > data_raw_hourly_temp.open), 1, np.nan))   # 9시는 무조건0, 매수시작하면 1
        data_raw_hourly_temp['status_ffill'] = data_raw_hourly_temp['status'].fillna(method='ffill')

        data_raw_hourly_temp['date'] = [day.strftime('%y%m%d') for day in data_raw_hourly_temp['date']]
        data_raw_hourly_temp['date'] = data_raw_hourly_temp['date'].shift(start_hour)

        # 시간대별 rolling max를 계산한 뒤 low가 이보다 작으면 low에 매도되는걸로 수익률 계산.
        data_raw_hourly_temp['max_price'] = data_raw_hourly_temp[['date','high']].groupby(by='date').cummax()  #일별 누적맥스
        data_raw_hourly_temp['max_price'].fillna(0,inplace=True)

        if 'pr' in str(stoploss):  #price_range
            data_raw_hourly_temp['price_vol'] = np.array(data_raw_hourly_temp['price_vol']) * float(stoploss[:-2])
        else:
            data_raw_hourly_temp['price_vol'] = float(stoploss)
#             gap = (data_raw_hourly_temp['max_price']-data_raw_hourly_temp['target_price'])/data_raw_hourly_temp['max_price']
#             data_raw_hourly_temp['price_vol'] = [0.05 if each_gap/2<0.05 else each_gap/2 for each_gap in gap]

        
        data_raw_hourly_temp['stoploss_price'] = data_raw_hourly_temp['max_price']*(1-data_raw_hourly_temp['price_vol'])
        # stoploss가격을 stoploss기준&시가-price_range*k
#         data_raw_hourly_temp['stoploss_price'] = [max(tp_low, sl) for tp_low, sl in zip(data_raw_hourly_temp['target_price_low'], data_raw_hourly_temp['stoploss_price'])]
        data_raw_hourly_temp['stoploss'] = np.where( data_raw_hourly_temp['low'] < data_raw_hourly_temp['stoploss_price'], 1, 0)

        data_raw_hourly_temp['status'] = np.where( data_raw_hourly_temp.stoploss.shift(1) == 1, 0, data_raw_hourly_temp.status )
        data_raw_hourly_temp['status_ffill'] = data_raw_hourly_temp['status'].fillna(method = 'ffill')

    # status가 0->1이면 target_price대비 수익률, stoploss가 0->1&status_ffill=1이면, 매도가대비 수익률, status_ffill이면 return, 나머지는 0
        data_raw_hourly_temp['strategy_return'] = np.where((data_raw_hourly_temp.status_ffill == 1)&(data_raw_hourly_temp.status_ffill.shift(1) == 0), 
                                                           np.log10(data_raw_hourly_temp.close/data_raw_hourly_temp.target_price), 
                                                           np.where((data_raw_hourly_temp.stoploss == 1)&(data_raw_hourly_temp.status_ffill == 1), 
                                                                    np.log10(data_raw_hourly_temp['stoploss_price']/data_raw_hourly_temp.open),
                                                                    np.where(data_raw_hourly_temp.status_ffill == 1, data_raw_hourly_temp['return'], 0) ) )

        dict_ohlcv = {'open':'first','high':'max','low':'min','close':'last','value':'sum','target_price':'first','price_vol':'first','return':'sum','strategy_return':'sum','stoploss':'sum','price_range':'first'}
        data_raw_temp = data_raw_hourly_temp[['open','high','low','close','value','price_range','target_price','price_vol','return','strategy_return','stoploss']].resample('24H', origin=start_datetime ).apply(dict_ohlcv)

    # price_vol : 종목의 가격흐름의 변동성%, target_vol : 거래의 제한 변동성%
        if target_vol == None:
            pass
        else:
#             data_raw_temp['price_vol'] = data_raw_temp['price_vol'].fillna(target_vol)
#             trading_portion = np.where(data_raw_temp['price_vol']/target_vol<1,1,data_raw_temp['price_vol']/target_vol)
            trading_portion = np.minimum(1, float(target_vol)/data_raw_temp['price_vol'])
#             np.where(target_vol/data_raw_temp['price_vol']>1,1,target_vol/data_raw_temp['price_vol'])
            
            data_raw_temp['strategy_return'] = np.log10(1+(_power10(data_raw_temp['strategy_return']) - 1)*trading_portion)

        buy_entry = np.where(data_raw_temp.strategy_return == 0, 0, 1)
        data_raw_temp['buy_entry'] = buy_entry
        data_raw_temp['buy_exit'] = buy_entry
        self.data_raw = data_raw_temp
        return data_raw_hourly_temp
       
    # data_raw에 stoploss를 고려한 return_adj, buy_entry, buy_exit 기입.
    def calculate_stoploss_logic(self):
        data_raw_temp = self.data_raw.copy()

        #이런식으로 상승가격대비 몇프로 하락하면 매도하는 식으로 stoploss 변경 가능.        
#         stoploss = (data_raw_temp.open_AT - data_raw_temp.low)/data_raw_temp.open_AT * 0.7
#         stoploss_signal = data_raw_temp.low_AT < data_raw_temp.open_AT*(1-stoploss.shift(1))
        
        stoploss = 0.02
        stoploss_signal = data_raw_temp.low_AT < data_raw_temp.open_AT*(1-stoploss)
        
        return_normal = np.log10(data_raw_temp.close/data_raw_temp.open_AT)
        
#         return_adjust = np.where(stoploss_signal, np.log10(data_raw_temp.open_AT*(1-stoploss.shift(1))/data_raw_temp.open_AT), return_normal)
        return_adjust = np.where(stoploss_signal, np.log10(data_raw_temp.open_AT*(1-stoploss)/data_raw_temp.open_AT), return_normal)
        
        data_raw_temp['strategy_return'] = return_adjust
        buy_trade_entry = np.where(data_raw_temp.open_AT > 0, 1, 0)
        buy_trade_exit = np.where(data_raw_temp.open_AT > 0, 1, 0)

        # buy_condition 적용
        buy_condition = buy_trade_entry     

        data_raw_temp['buy_entry'] = buy_trade_entry
        data_raw_temp['buy_exit'] = buy_trade_exit
        self.data_raw = data_raw_temp
#         return data_raw_temp

    def apply_price_noise_filter(self, noise_value = 0.2):
        data_raw_temp = self.data_raw.copy()
        noise = cal_ma((data_raw_temp.high-data_raw_temp.close)/(data_raw_temp.high - data_raw_temp.low),method='sma',length=5)
        data_raw_temp.buy_entry = np.where(noise.shift(1)<noise_value,data_raw_temp.buy_entry, 0)
        data_raw_temp.buy_exit = data_raw_temp.buy_entry
        data_raw_temp.strategy_return = np.where(data_raw_temp.buy_entry,data_raw_temp.strategy_return,0)
        self.data_raw = data_raw_temp
    def apply_volume_filter(self, target_volume = 100000000000):
        data_raw_temp = self.data_raw.copy()
#         volume_filter = (data_raw_temp.value > 10**11) # 매수할 수 있으면 True
        volume_filter = (data_raw_temp.value > target_volume) # 매수할 수 있으면 True
        data_raw_temp.buy_entry = np.where(volume_filter.shift(1), data_raw_temp.buy_entry, 0)
        data_raw_temp.buy_exit = data_raw_temp.buy_entry
        data_raw_temp.strategy_return = np.where(data_raw_temp.buy_entry,data_raw_temp.strategy_return,0)
        self.data_raw = data_raw_temp
    def apply_volume_trend_filter(self, method = 'sma', length = 5):
        data_raw_temp = self.data_raw.copy()
#         volume_trend_filter = (data_raw_temp['volume'] > ta.sma(data_raw_temp['volume'], length=5, talib=False))
        volume_trend_filter = (data_raw_temp['volume'] > cal_ma(data_raw_temp['volume'], method=method, length=length))
        
        data_raw_temp.buy_entry = np.where(volume_trend_filter.shift(1), data_raw_temp.buy_entry, 0)
        data_raw_temp.buy_exit = data_raw_temp.buy_entry
        data_raw_temp.strategy_return = np.where(data_raw_temp.buy_entry,data_raw_temp.strategy_return,0)
        self.data_raw = data_raw_temp
    def apply_price_trend_filter(self, method = 'sma', length = 5):
        data_raw_temp = self.data_raw.copy()
        price_trend_filter = (data_raw_temp['close'] > cal_ma(data_raw_temp['close'], method=method, length=length)*0.95)
        data_raw_temp.buy_entry = np.where(price_trend_filter.shift(1), data_raw_temp.buy_entry, 0)
        data_raw_temp.buy_exit = data_raw_temp.buy_entry
        data_raw_temp.strategy_return = np.where(data_raw_temp.buy_entry,data_raw_temp.strategy_return,0)
        self.data_raw = data_raw_temp
        
    def calculate(self):
        for strategy in self.strategy_list:
            if 'base' in strategy:
                if len(strategy.split(', ')) == 1:
                    self.calculate_volatility_breakout()
#                     self.calculate_turtle()
                else:
                    _, target_vol = strategy.split(', ')
                    self.calculate_volatility_breakout(float(target_vol))
            elif 'ml' in strategy:
                self.calculate_ml()
            elif 'stoploss' in strategy:
                if len(strategy.split(', ')) == 1:
                    self.calculate_volatility_breakout_stoploss()
                elif len(strategy.split(', ')) == 2:
                    _, target_stoploss = strategy.split(', ')
                    self.calculate_volatility_breakout_stoploss(target_stoploss)
                elif len(strategy.split(', ')) == 3:
                    _, target_stoploss, target_vol = strategy.split(', ')
                    self.calculate_volatility_breakout_stoploss(target_stoploss, target_vol)
                else:
                    _, target_stoploss, target_vol, start_time = strategy.split(', ')
                    self.calculate_volatility_breakout_stoploss(target_stoploss, target_vol, int(start_time))
#                 self.calculate_stoploss_logic()
            elif 'filter' in strategy:
                if 'volume_filter' in strategy:
                    if len(strategy.split(', ')) == 1:
                        self.apply_volume_filter()
                    else:
                        _, target_volume = strategy.split(', ')
                        self.apply_volume_filter(int(target_volume))
                elif 'volume_trend_filter' in strategy:
                    if len(strategy.split(', ')) == 1:
                        self.apply_volume_trend_filter()
                    else:
                        _, method, length = strategy.split(', ')
                        self.apply_volume_trend_filter(method, int(length))
                elif 'price_trend_filter' in strategy:
                    if len(strategy.split(', ')) == 1:
                        self.apply_price_trend_filter()
                    else:
                        _, method, length = strategy.split(', ')
                        self.apply_price_trend_filter(method, int(length))
                elif "price_noise_filter" in strategy:
                    if len(strategy.split(', ')) == 1:
                        self.apply_price_noise_filter()
                    else:
                        _, noise_value = strategy.split(', ')
                        self.apply_price_noise_filter(float(noise_value))
            elif strategy == 'None':
                continue
            else:
                raise ValueError('Strategy Name 확인 필요')
#         return self.data_raw[['open','close','high','low','target_price','stoploss','strategy_return','return','buy_entry','buy_exit']]        
        return self.data_raw[['strategy_return','return','buy_entry','buy_exit']]

In [101]:
def define_data ( start , end , code, timeframe = 'daily', time_start = 9 ) :
    if timeframe == 'daily':
        if time_start == 9:
            data_all = pd.read_pickle("./data/data_coin_daily.pkl")
            data_all = data_all[data_all.coin == code]
            data_all = data_all[(data_all.date >= start)&(data_all.date <= end)]
        else:
            data_hourly = pd.read_pickle('./data/data_coin_hourly.pkl')
            data_hourly = data_hourly[data_hourly.coin == code]
            start_datetime = start.replace(hour=time_start)
            data_hourly = data_hourly[data_hourly.date >= start_datetime]
            data_hourly.index = data_hourly.date
            dict_ohlcv = {'open':'first','high':'max','low':'min','close':'last','value':'sum','volume':'sum','date':'first'}
            data_all = data_hourly.resample('24H',origin=start_datetime).apply(dict_ohlcv)
            data_all = data_all[(data_all.date >= start)&(data_all.date <= end)]
    elif timeframe == 'hourly':
        data_all = pd.read_pickle('./data/data_coin_hourly.pkl')
        data_all = data_all[data_all.coin == code]
        data_all = data_all[(data_all.date >= start)&(data_all.date <= end)]
    else:
        print("Timeframe 확인")
    data_all['return'] = np.log10(data_all.close / data_all.close.shift(1))
    return data_all.reset_index(drop=True)



In [105]:
date_start1 = datetime(2017,11,1)
# date_start2 = datetime(2022,1,1)
# date_end1 = datetime(2021,12,31)
date_end2 = datetime(2022,8,31)
coin_code = 'KRW-BTC'
start_time = 9

# base 전략

# for date_start, date_end in [(date_start1, date_end2), (date_start1, date_end1), (date_start2, date_end2)]:
data_raw = define_data(date_start1-relativedelta(days=10),date_end2,coin_code,time_start=start_time,timeframe='hourly')

case2 = Backtest(data_raw, Strategy_Volatility, strategy_list=[f'stoploss'])
case2.trading_fee = 0.1
case2.cal_strategy()
case2.test_result = case2.test_result.loc[date_start1:date_end2]
case2.cal_result()
print(case2.analyze_result_cont())
print(case2.analyze_result_ind())

전체수익 : -41.84 %, CAGR : -10.61 %, MDD : -53.46 %, STD : 7.36 %, Sharpe : -1.44
거래횟수 (수익, 승률 ) : 606 ( 257, 0.42), 평균 수익률 % (수익, 손실, 손익비) : -0.08 (1.24, -1.05, 1.18)


In [109]:
case1.data_hr

,open,high,low,close,volume,value,start,buy_price,gap_price,signal,signal_fill,open_adj,status,status_fill,close_adj,return,strategy_return
2017-10-22 09:00:00,NaN,7060000.0,6709000.0,7039000.0,4.391016,3.005405e+07,1,NaN,NaN,0.0,0.0,NaN,0.0,0.0,7039000.0,NaN,NaN
2017-10-23 09:00:00,7039000.0,7100000.0,6668000.0,6971000.0,14.039344,9.705252e+07,1,NaN,NaN,0.0,0.0,7039000.0,0.0,0.0,6971000.0,-0.004216,-0.000000
2017-10-24 09:00:00,6971000.0,7052000.0,6350000.0,6479000.0,98.674229,6.513772e+08,1,7187000.0,216000.0,0.0,0.0,6971000.0,0.0,0.0,6479000.0,-0.031787,-0.000000
2017-10-25 09:00:00,6479000.0,6644000.0,6212000.0,6643000.0,108.363974,6.959687e+08,1,6971000.0,492000.0,0.0,0.0,6479000.0,0.0,0.0,6643000.0,0.010856,0.000000
2017-10-26 09:00:00,6643000.0,6873000.0,6534000.0,6811000.0,428.508520,2.881468e+09,1,6859000.0,216000.0,1.0,1.0,6859000.0,1.0,1.0,6811000.0,0.010847,-0.003050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-13 09:00:00,32368000.0,32957000.0,32260000.0,32562000.0,4388.330322,1.431605e+11,1,32966000.0,598000.0,0.0,0.0,32368000.0,0.0,0.0,32562000.0,0.002595,0.000000
2022-08-14 09:00:00,32562000.0,33197000.0,32310000.0,32544000.0,6387.198266,2.089719e+11,1,32910500.0,348500.0,1.0,1.0,32910500.0,1.0,1.0,32544000.0,-0.000240,-0.004864
2022-08-15 09:00:00,32544000.0,33450000.0,32139000.0,32445000.0,6914.807593,2.260089e+11,1,32987500.0,443500.0,1.0,1.0,32544000.0,1.0,1.0,32445000.0,-0.001323,-0.001323
2022-08-16 09:00:00,32445000.0,32600000.0,31865000.0,32082000.0,4179.746422,1.346492e+11,1,33100500.0,655500.0,0.0,0.0,32445000.0,0.0,0.0,32082000.0,-0.004886,-0.000000


In [108]:
case2.test_result

,strategy_return,return,buy_entry,buy_exit
2017-11-01 09:00:00,0.006679,0.021462,1,1
2017-11-02 09:00:00,0.000000,0.022730,0,0
2017-11-03 09:00:00,0.000000,-0.003929,0,0
2017-11-04 09:00:00,-0.001506,0.010694,1,1
2017-11-05 09:00:00,0.000000,0.005532,0,0
...,...,...,...,...
2022-08-26 09:00:00,-0.006984,-0.020191,1,1
2022-08-27 09:00:00,0.000000,-0.003586,0,0
2022-08-28 09:00:00,0.000000,-0.010138,0,0
2022-08-29 09:00:00,0.004713,0.010967,1,1
